# Прогнозирование временных рядов

Прогнозирование временных рядов — это важная задача в аналитике данных, которая помогает предсказывать будущее поведение данных на основе исторических данных. В этом блоке мы рассмотрим основы анализа временных рядов, изучим, как работать с ними в Python, и познакомимся с основными методами оценки точности прогнозов.

---

## Введение в анализ временных рядов

### Что такое временные ряды и их использование

**Временные ряды** — это последовательность наблюдений, записанных через равные промежутки времени. Они широко используются для анализа данных, которые зависят от времени, таких как прогнозирование продаж, температуры, финансовых показателей, спроса на продукцию и т.д.

Примеры применения временных рядов:
- **Финансовый анализ**: прогнозирование цен на акции или обменных курсов.
- **Производственные процессы**: прогнозирование объёмов выпуска продукции.
- **Экономика**: анализ и прогнозирование макроэкономических показателей.
- **Прогнозирование спроса**: планирование запасов на основе данных о продажах.

### Основные методы прогнозирования

1. **Простое сглаживание (Simple Moving Average)**:
   - Метод заключается в усреднении ряда данных за определённый промежуток времени.
   - Пример: среднее значение продаж за последние 3 месяца.

2. **Экспоненциальное сглаживание (Exponential Smoothing)**:
   - Усреднение данных с учётом весов, придаваемых более свежим данным.
   - Пример: модель **Holt-Winters**, которая учитывает как тренд, так и сезонность данных.

3. **ARIMA (AutoRegressive Integrated Moving Average)**:
   - Популярная модель для прогнозирования временных рядов, которая включает авторегрессию (AR), интегрированную компоненту (I) и скользящее среднее (MA).
   - Применяется для прогнозирования данных с трендами и сезонными колебаниями.

4. **Prophet**:
   - Модель, разработанная Facebook, специально для работы с временными рядами. Хорошо справляется с данными, содержащими тренды, сезонность и аномалии.

5. **TSFresh и Pmdarima**:
   - **TSFresh** используется для извлечения признаков из временных рядов для дальнейшего анализа и машинного обучения.
   - **Pmdarima** автоматически подбирает параметры ARIMA и выполняет прогнозирование.

---

## Работа с временными рядами в Python

#### Введение в библиотеки Pandas для работы с временными рядами

**Pandas** предоставляет множество удобных инструментов для работы с временными рядами. Среди них:
- Индексация по дате и времени.
- Вычисление скользящего среднего.
- Ресемплирование данных (например, агрегирование по дням, неделям, месяцам).

##### Пример работы с временными рядами в Pandas:

```python
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_excel('timeseries.xlsx', index_col='date', parse_dates=['date'])

# Вывод первых строк с индексом времени
df.head()
```

Вывод графика

```python
# Построение графика временного ряда
df['value'].plot(title='Объем, тыс. тонн', figsize=(17,4))
```

### Пояснение:
- **`parse_dates=['Date']`** — преобразует столбец с датами в формат `datetime`.
- **`index_col='Date'`** — делает столбец с датами индексом DataFrame.
- **`plot()`** — построение графика временного ряда.

## Поиск выбросов во временных рядах с учётом сезонных колебаний

Временные ряды могут иметь как случайные выбросы, так и выбросы, связанные с сезонными колебаниями. Выбросы, не принимающие во внимание сезонность, могут ошибочно считаться аномалиями. В этом блоке рассмотрим, как искать выбросы во временных рядах с учётом сезонных колебаний и как корректировать их.

### Шаг 1: Поиск выбросов с учётом сезонности

Для корректного поиска выбросов во временных рядах с выраженной сезонностью часто применяются специализированные методы. Вот несколько подходов:

1. **Декомпозиция временного ряда** (с использованием библиотек **statsmodels** или **Prophet**) позволяет отделить сезонную составляющую от тренда и случайных колебаний.
2. **Использование скользящего среднего с учётом сезонных изменений**.
3. **Поиск выбросов с помощью локальной регрессии (LOESS)**.

#### 1.1 Декомпозиция временного ряда

Декомпозиция позволяет разделить временной ряд на три компонента:
- **Тренд** — долгосрочные изменения во времени.
- **Сезонность** — повторяющиеся циклические колебания.
- **Остатки** — случайные флуктуации и выбросы.

Сначала разделим временной ряд с помощью декомпозиции, а затем поищем выбросы в остатках.

```python

import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Декомпозиция временного ряда (добавьте свой временной ряд в 'Cost_USD')
decomposition = seasonal_decompose(df['value'], model='additive', period=12)

# Визуализация компонентов декомпозиции
decomposition.plot()
plt.show()

# Выделение компоненты остатков (residuals)
residuals = decomposition.resid

# Определение выбросов в остатках с помощью метода межквартильного размаха (IQR)
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Поиск выбросов в остатках
outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]

# Вывод выбросов
print('Выбросы:')
outliers
```

### Пояснение:
- **`seasonal_decompose()`** — функция для декомпозиции временного ряда на тренд, сезонность и остатки. 
- **Остатки** содержат случайные флуктуации и выбросы, поэтому именно они используются для поиска аномалий.

#### 1.2 Скользящее среднее с учётом сезонных изменений

Другим подходом к поиску выбросов является использование скользящего среднего с учётом сезонных колебаний. Важно выбрать подходящий размер окна, который соответствует длине сезонного цикла.

```python
# Скользящее среднее с окном, равным длине сезонного цикла (например, 12 месяцев)
rolling_mean = df['value'].rolling(window=12).mean()
rolling_std = df['value'].rolling(window=12).std()

# Поиск выбросов на основе скользящего среднего и стандартного отклонения
outliers = df[(df['value'] > rolling_mean + 2 * rolling_std) | 
              (df['value'] < rolling_mean - 2 * rolling_std)]

# Вывод выбросов
print(outliers[['value']])
```

### Пояснение:
- **`rolling(window=12)`** — скользящее среднее и стандартное отклонение с окном, равным 12 наблюдениям (например, 12 месяцев для годового цикла).

#### 1.3 Поиск выбросов с использованием локальной регрессии (LOESS)

Локальная регрессия (LOESS) — это метод, который позволяет сгладить данные с учётом их локальной структуры, включая сезонность. Этот метод помогает найти выбросы, которые отклоняются от локальных трендов.

```python
from statsmodels.nonparametric.smoothers_lowess import lowess

# Применение метода LOESS для сглаживания временного ряда
smoothed = lowess(df['value'], df.index, frac=0.1)

# Вычисление остатков (разница между исходными данными и сглаженными)
residuals = df['value'] - smoothed[:, 1]

# Поиск выбросов в остатках с использованием метода IQR
Q1 = pd.Series(residuals).quantile(0.25)
Q3 = pd.Series(residuals).quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]

# Вывод выбросов
outliers
```

### Пояснение:
- **`lowess()`** — сглаживает временной ряд с помощью локальной регрессии.
- Выбросы затем находятся в остатках, используя метод IQR.

### Шаг 2: Рекомендации по исправлению выбросов

#### 2.1 Коррекция выбросов с учётом сезонных колебаний

##### 2.1.1 Удаление выбросов в остатках

Если выбросы являются результатом случайных аномалий и не имеют важного значения для тренда или сезонности, их можно удалить.

```python
# Удаление выбросов в остатках
df_cleaned = df[(residuals >= lower_bound) & (residuals <= upper_bound)]
```

#### Рекомендации:
- Удаляйте выбросы в остатках после учёта сезонности и тренда, чтобы не потерять важные сезонные или трендовые колебания.

#### 2.1.2 Замена выбросов на сезонные значения

Если выбросы являются аномальными отклонениями, но сами данные важны для анализа, можно заменить их на соответствующие сезонные значения (или на среднее для каждого сезона).

```python
# Замена выбросов на среднее сезонное значение
seasonal_mean = decomposition.seasonal.mean()
df['value_corrected'] = df['value'].apply(lambda x: seasonal_mean if (x < lower_bound or x > upper_bound) else x)
```

#### Рекомендации:
- Замена выбросов сезонными значениями сохраняет целостность данных и учитывает сезонные колебания.

#### 2.2 Использование логарифмического преобразования

Логарифмическое преобразование может уменьшить влияние выбросов, особенно когда выбросы значительно отклоняются по величине.

```python
# Применение логарифмического преобразования к данным
df['Log_value'] = np.log(df['value'])
```

#### Рекомендации:
- Логарифмическое преобразование часто помогает, если выбросы пропорциональны значению переменной и связаны с изменением масштаба.

### Заключение

Для поиска выбросов в сезонных временных рядах лучше всего использовать методы, которые учитывают структуру данных, такие как декомпозиция или скользящее среднее с сезонным окном. Исправление выбросов может включать удаление аномалий, замену сезонными средними значениями или использование логарифмического преобразования для сглаживания отклонений. 

1. **Декомпозиция** и **LOESS** — позволяют точно выделить остатки для поиска аномалий.
2. **Скользящее среднее с учётом сезонности** — позволяет находить выбросы в сезонных рядах.
3. **Исправление**: удаление, замена на сезонные значения или логарифмическое преобразование.

Выбор метода зависит от задачи и структуры данных.

## Использование библиотеки Prophet для прогнозирования

Prophet — это мощная библиотека для прогнозирования временных рядов, которая автоматически учитывает тренды и сезонность данных.

##### Установка Prophet:

```bash
!pip -q install prophet
```

##### Пример использования Prophet:

```python
from prophet import Prophet

# Подготовка данных для Prophet
df_prophet = df[['date', 'value']].reset_index().rename(columns={'date': 'ds', 'value': 'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Создание модели Prophet
model = Prophet()
model.fit(df_prophet)

# Прогнозирование на следующие 30 дней
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# Визуализация прогноза
model.plot(forecast)
plt.show()
```

### Пояснение:
- **`ds`** — стандартное имя для столбца с датами в Prophet.
- **`y`** — значение временного ряда, которое нужно прогнозировать.
- **`make_future_dataframe(periods=30)`** — создаёт прогноз на следующие 30 дней.

## Использование библиотеки Pmdarima для ARIMA

**Pmdarima** автоматизирует процесс выбора параметров для модели ARIMA.

##### Установка Pmdarima:

```bash
!pip install -q pmdarima
```

##### Пример использования Pmdarima:

```python
from pmdarima import auto_arima

# Автоматический подбор параметров ARIMA
model_arima = auto_arima(df['value'], seasonal=True, m=12)

# Прогнозирование на следующие 30 периодов
forecast_arima = model_arima.predict(n_periods=12)

# Вывод прогноза
print(forecast_arima)
```

### Пояснение:
- **`auto_arima()`** — автоматически подбирает параметры модели ARIMA.
- **`seasonal=True`** и **`m=12`** — задают сезонность данных с периодом 12 (например, месячные данные).

## Оценка моделей временных рядов

После построения модели важно оценить её точность, используя различные метрики. В прогнозировании временных рядов наиболее часто применяются следующие метрики:

1. **MAE (Mean Absolute Error)** — средняя абсолютная ошибка. Измеряет среднее абсолютное отклонение прогнозов от фактических значений.

   Формула:
   $$
   MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
   $$

2. **RMSE (Root Mean Squared Error)** — среднеквадратичная ошибка. Оценка средней ошибки прогнозирования с увеличенным влиянием больших отклонений.

   Формула:
   $$
   RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
   $$

3. **MAPE (Mean Absolute Percentage Error)** — средняя абсолютная процентная ошибка. Показывает, на сколько процентов в среднем прогноз отличается от фактического значения.

   Формула:
   $$
   MAPE = \frac{1}{n} \sum_{i=1}^{n} \left( \frac{|y_i - \hat{y}_i|}{y_i} \right) \times 100
   $$

#### Пример расчёта метрик:

```python
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# Истинные значения и прогнозы (например, для Prophet)
y_true = df['value'][-30:].values  # последние 30 значений для оценки
y_pred = forecast['yhat'][-30:].values  # прогнозируемые значения Prophet

# MAE
mae = mean_absolute_error(y_true, y_pred)

# RMSE
rmse = root_mean_squared_error(y_true, y_pred)

# MAPE
mape = (abs((y_true - y_pred) / y_true)).mean() * 100

# Вывод метрик
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')
```

### Пояснение:
- **`mean_absolute_error()`** — вычисляет среднюю абсолютную ошибку.
- **`mean_squared_error(squared=False)`** — вычисляет RMSE.
- **`MAPE`** — рассчитывается вручную, поскольку она не встроена в `sklearn`.

## Заключение

1. **Основы временных рядов**: Временные ряды используются для анализа данных, зависящих от времени, с использованием различных методов прогнозирования (от простого сглаживания до сложных моделей).
2. **Работа с временными рядами в Python**: Мы рассмотрели библиотеки Pandas для работы с временными рядами и такие инструменты для прогнозирования, как Prophet и Pmdarima.
3. **Оценка точности моделей**: MAE, RMSE и MAPE — основные метрики, используемые для оценки качества прогноза.

Эти знания помогут вам эффективно анализировать и прогнозировать временные ряды.

# Задание

Проанализируйте, очистите ряд и постройте прогноз двумя методами. 

Сравните результаты и выберите наиболее точный. 